In [1]:
import json
import http.client
from bs4 import BeautifulSoup
import yagmail

In [2]:
API_KEY = "288a2c27-66a8-443a-9028-722bab20f3aa"
SENDER_EMAIL = "angelpeters15@gmail.com"
APP_PASSWORD = "xgaa umyz mxzx wowg"

In [3]:
def fetch_jooble_jobs(api_key, job_titles, limit=20, location="New York", excluded_keywords=None):
    host = 'jooble.org'
    all_jobs = []
    excluded_keywords = excluded_keywords or []

    for job_title in job_titles:
        print(f"Fetching jobs for: {job_title}...", flush=True)  # Progress indicator
        body = json.dumps({"keywords": job_title, "location": location})
        connection = http.client.HTTPConnection(host)
        headers = {
            "Content-type": "application/json",
            "Authorization": f"Bearer {api_key}"
        }

        try:
            connection.request('POST', f'/api/{api_key}', body, headers)
            response = connection.getresponse()

            if response.status == 200:
                data = json.loads(response.read().decode('utf-8'))
                jobs = data.get("jobs", [])
                for job in jobs:
                    title = job.get("title", "").strip()
                    if not title or any(keyword.lower() in title.lower() for keyword in excluded_keywords):
                        continue
                    company = job.get("company", "No Company")
                    location = job.get("location", "No Location")
                    link = job.get("link", "No Link")
                    snippet = BeautifulSoup(job.get("snippet", "No Description"), "html.parser").get_text()
                    job_details = f"{title} at {company} - {location}: {link}\nDescription: {snippet}\n"
                    print(job_details, flush=True)  # Immediate printing

                    all_jobs.append(job_details)
                    if len(all_jobs) >= limit:
                        break
            else:
                print(f"Error fetching jobs for {job_title}: {response.status} - {response.reason}")

        except Exception as e:
            print(f"Error during API request for {job_title}: {e}")

        finally:
            connection.close()

    return all_jobs

           

In [4]:
def send_email(sender, app_password, receiver, subject, jobs, receiver_name=None):
    body = (
        f"Good Morning {receiver_name or ''},\n\n"
        "Here are the latest job openings for Electrical Engineering:\nI advise applying directly on the company website instead of clicking the link.\n"
        + "\n".join(jobs) 
    )
    yag = yagmail.SMTP(sender, app_password)
    yag.send(to=receiver, subject=subject, contents=body)
    print("Email sent successfully!")

In [5]:
def job_search(api_key, sender, app_password, receiver, receiver_name, subject, job_titles, excluded_keywords, limit=20, location="New York"):
    jobs = fetch_jooble_jobs(api_key, job_titles, limit, location, excluded_keywords)
    if jobs:
        send_email(sender, app_password, receiver, subject, jobs, receiver_name)
    else:
        print("No jobs found.")

In [ ]:
job_configs = [
    
    {
        "receiver": "emmanuelijasuyi@gmail.com",
        "receiver_name": "Emmanuel",
        "subject": "Morning Job Listings for Electrical Engineering",
        "job_titles": ["Electrical Engineer", "Entry Level Engineer"],
        "excluded_keywords": ["Senior", "Lead", "Manager", "Director", "Principal", "Sr.", "Civil"],
    },
    {
        "receiver": "stephonie@aol.com",
        "receiver_name": "Stephonie",
        "subject": "Mid-Level and Assistant Manager Job Listings",
        "job_titles": ["Electrical Engineer", "Mid Level Engineer", "Assistant Manager Engineer"],
        "excluded_keywords": ["Entry Level"],
    },
    
     {
        "receiver": "angelpeters15@gmail.com",
        "receiver_name": "Angel",
        "subject": "Morning Job Listings for Software Engineering/Data Science",
        "job_titles": ["Entry Level Software Engineer", "Junior Software Engineer","Junior Data Scientist"],
        "excluded_keywords": ["Senior", "Lead", "Manager", "Director", "Principal", "Sr.", "Civil"],
    },
    {
        "receiver": "liza.peters1@yahoo.com",
        "receiver_name": "Liza",
        "subject": "Morning Job Listings for Graphic Design",
        "job_titles": ["Graphic Designer", "Digital Marketing Specialist", "Public Relations Specialist"],
        "excluded_keywords": ["Senior", "Lead", "Manager", "Director", "Principal", "Sr.", "Civil"],
    },



]

In [7]:
for config in job_configs:
    job_search(
        api_key=API_KEY,
        sender=SENDER_EMAIL,
        app_password=APP_PASSWORD,
        receiver=config["receiver"],
        receiver_name=config["receiver_name"],
        subject=config["subject"],
        job_titles=config["job_titles"],
        excluded_keywords=config["excluded_keywords"]
    )

Fetching jobs for: Electrical Engineer...
Graduate Electrical Engineer (Available 2025) at Arup - New York, NY: https://jooble.org/jdp/6012017862319769463
Description:  ...using Revit MEP 
 Contribute to the development of reports and electrical calculations 
 Work with people from other disciplines and/...  ...’d like to hear from you. 
 Bachelor’s Degree in Electrical Engineering (BSEE), or similar degree, bachelor’s degree in Architectural... 

Tenure-Track Faculty, Department of Electrical and Computer Engineering, New York University, Brooklyn, NY | New York University at New York University - Brooklyn, NY: https://jooble.org/jdp/-9194656051950560499
Description:  ...The  Department of Electrical and Computer Engineering  at the NYU Tandon School of Engineering invites applications for two faculty positions at the tenure-track Assistant Professor rank, to start on September 1, 2025. Senior candidates with outstanding scholarly achievements... 

Electrical Engineer at Leidos - New 